In [1]:
import re

import pandas as pd

In [2]:
podrobno_df = pd.read_csv('/kaggle/input/podrobno-uz-dataset/podrobno_uz_translated_cleaned_labeled_compact.csv')
uzreport_df = pd.read_csv('/kaggle/input/uzreport-uz-dataset/uzreport_uz_cleaned_labeled_compact.csv')
gazeta_df = pd.read_csv('/kaggle/input/uznews-preprocessed-ready-for-training/uznews_preprocessed_labeled.csv')

In [3]:
podrobno_df.head()

,text,date,sentiment,promotional
0,Mirziyoyev Germaniyaga tashrif buyurdi \n\nU ...,2023-05-02 13:23:49,negative,no
1,O'zbekistonda yana bir dam olish kuni bo'ladi...,2023-05-02 12:26:41,positive,no
2,O'zgidromet kelgusi may oyi uchun ob-havo pro...,2023-05-02 12:01:42,positive,no
3,Xitoyning CC7 Industries kompaniyasi Toshkent...,2023-05-02 11:29:48,positive,no
4,"Toshkentda yana bir baxtsiz hodisa yuz berdi,...",2023-05-02 11:04:59,negative,no


In [4]:
uzreport_df.head()

,text,date,sentiment,promotional
0,Finlyandiya o‘z hududida Amerika harbiy bazasi...,2023-05-02 12:31:58,negative,no
1,Bolgariya Bosh prokurori Ivan Geshevning hayot...,2023-05-02 12:02:00,positive,no
2,"""PSJ"" Messining o'rniga bir nechta nomzodlar k...",2023-05-02 11:45:48,positive,no
3,Endi 1-may Konstitutsiya kuni sifatida nishonl...,2023-05-02 11:31:41,negative,no
4,O'zbekistonda 3 oy ichida umumiy ovqatlanish k...,2023-05-02 11:15:41,positive,no


In [5]:
gazeta_df.head()

,text,clean_text,sentiment
0,O'zbekiston futbol assotsiatsiyasi intizom qo'...,ozbekiston futbol assotsiatsiyasi intizom qomi...,positive
1,Rustam Yusupov Toshkent obodonlashtirish Bosh ...,rustam yusupov toshkent obodonlashtirish bosh ...,positive
2,Xitoyning Yutong kompaniyasi yil oxirigacha Sa...,xitoyning yutong kompaniyasi yil oxirigacha sa...,positive
3,Samarqand viloyatida 3 gektar o'rmon xo'jaligi...,samarqand viloyatida gektar ormon xojaligi yer...,positive
4,Andijon viloyati Xonobodda bir kishi davlat or...,andijon viloyati xonobodda bir kishi davlat or...,negative


# Data cleaning

In [6]:
podrobno_df.columns

Index(['text', 'date', 'sentiment', 'promotional'], dtype='object')

In [7]:
podrobno_df.shape

(7532, 4)

In [8]:
# consider only non-promotional news
podrobno_df = podrobno_df[podrobno_df['promotional'] == 'no'] 
uzreport_df = uzreport_df[uzreport_df['promotional'] == 'no'] 

print(podrobno_df.shape)
print(uzreport_df.shape)

(7229, 4)
(19687, 4)


In [9]:
pd.set_option('display.max_colwidth', None)

podrobno_df.sample(10)['text']

434                                                                                                                                                      O'zbekistonda hayvonlarga nisbatan shafqatsizlikning yana bir dahshatli holati qayd etildi. Buxoro viloyatida yashovchi burro quloqlarini vahshiyona kesib tashladi va hayvonni ko'chada qonga tashlab qo'ydi\n\n
6514                                                        Davlat xavfsizlik xizmati (GBS) xodimlari Toshkentning bir nechta bozorlarida rahbar xodimlar tomonidan tovlamachilik va firibgarlik holatlarini aniqladilar.\n\nXususan, Yangiobod ixtisoslashtirilgan bozorining mas'ul xodimlaridan biri tadbirkorlik bilan shug'ullanmoqchi bo'lgan fuqarodan talab qildi,
1575                                                                                                                                                                                                                                                 O'zbekiston fuqarosi Turkiyad

In [10]:
uzreport_df.sample(10)['text']

15983    Yil boshidan beri hokim yordamchilari tomonidan 14 ta jinoyat sodir etilgan\n\nBu kabi holatlarni oldini olish maqsadida hokim yordamchilarini halollik bo‘yicha maxsus kurslarda o‘qitish tizimi yo‘lga qo‘yildi\n\n https://uzreport.news/society/yil-boshidan-beri-hokim-yordamchilari-tomonidan-14-ta-jinoyat-sodir-etilgan\n\n
5751                                                                                                                                      O'zbekistonliklar dam olish uchun tanlagan davlatlarining TOP-5 ro'yxati e'lon qilindi\n\n https://uzreport.news/society/ozbekistonliklar-dam-olish-tanlagan-top-5-davlat-royxati-elon-qilindi\n\n
1416                                                                                                      Avstraliya MOQ rahbari rossiyalik sportchilarni Olimpiada-2032da qabul qilishga tayyorligini aytdi\n\n https://uzreport.news/sports/avstraliya-moq-rahbari-rossiyalik-sportchilarni-olimpiada-2032da-qabul-qilishga-tayyorligi\n\n
1

## clean function

In [11]:
# final version
def clean_text(text):
    if not isinstance(text, str):
        return ''

    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove digits
    text = re.sub(r'\d+', '', text)
    
    # exception: replace / - _ with space
    text = re.sub(r'/', ' ', text)
    text = re.sub(r'[-_]', ' ', text)
    
    # Replace all ' with ‘ except for cases where it appears as a single character by itself
    text = re.sub(r"(?<!\w)'(?!\w)|'", '‘', text)
        
    # Remove Russian letters
    text = re.sub(r'[А-Яа-я]+', '', text)

    # Remove all non-word characters except for ‘ and convert to lowercase
    text = re.sub(r"[^\w\s‘’]+(?<!‘)", "", text).lower()

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

## apply cleaning
gazeta_df
podrobno_df
uzreport_df

In [12]:
gazeta_df['cleaned_text'] = gazeta_df['text'].apply(clean_text).copy()
podrobno_df['cleaned_text'] = podrobno_df['text'].apply(clean_text).copy()
uzreport_df['cleaned_text'] = uzreport_df['text'].apply(clean_text).copy()

In [13]:
gazeta_df = gazeta_df[['cleaned_text', 'sentiment']]
podrobno_df = podrobno_df[['cleaned_text', 'sentiment']]
uzreport_df = uzreport_df[['cleaned_text', 'sentiment']]

gazeta_df = gazeta_df.rename(columns={'cleaned_text': 'text'})
podrobno_df = podrobno_df.rename(columns={'cleaned_text': 'text'})
uzreport_df = uzreport_df.rename(columns={'cleaned_text': 'text'})

In [14]:
# check for empty strings
print((podrobno_df['text'].str.strip().str.len() == 0).sum())

11


In [15]:
podrobno_df.drop(podrobno_df[podrobno_df['text'].str.strip().str.len() == 0].index, inplace=True)

In [16]:
# check again
print((podrobno_df['text'].str.strip().str.len() == 0).sum())

0


## processing long texts: truncation

In [17]:
uzreport_df['length'] = uzreport_df['text'].str.len()
print(uzreport_df['length'].describe())

count    19687.000000
mean       138.232082
std         97.372912
min          7.000000
25%         78.000000
50%        101.000000
75%        156.000000
max        446.000000
Name: length, dtype: float64


In [18]:
podrobno_df['length'] = podrobno_df['text'].str.len()
print(podrobno_df['length'].describe())

count    7218.000000
mean      206.240371
std        97.633875
min        19.000000
25%       121.000000
50%       183.000000
75%       302.000000
max       456.000000
Name: length, dtype: float64


In [19]:
gazeta_df['length'] = gazeta_df['text'].str.len()
print(gazeta_df['length'].describe())

count    3825.000000
mean      266.758954
std       120.210293
min        48.000000
25%       190.000000
50%       254.000000
75%       317.000000
max      1964.000000
Name: length, dtype: float64


In [20]:
long_messages = gazeta_df[gazeta_df['length'] > 450]['text']
long_messages.nunique()

223

In [21]:
max_length = 450

gazeta_df['text'] = gazeta_df['text'].apply(lambda x: x[:max_length] if len(x) > max_length else x)

In [22]:
gazeta_df['length'] = gazeta_df['text'].str.len()
print(gazeta_df['length'].describe())

count    3825.000000
mean      258.461438
std        90.882345
min        48.000000
25%       190.000000
50%       254.000000
75%       317.000000
max       450.000000
Name: length, dtype: float64


## saving preprocessed datasets

In [23]:
gazeta_df = gazeta_df[['text', 'sentiment']]
podrobno_df = podrobno_df[['text', 'sentiment']]
uzreport_df = uzreport_df[['text', 'sentiment']]

In [24]:
gazeta_df.to_csv('gazetauz_preprocessed_ready_for_modeling.csv', index=False)
podrobno_df.to_csv('podrobnouz_preprocessed_ready_for_modeling.csv', index=False)
uzreport_df.to_csv('uzreportuz_preprocessed_ready_for_modeling.csv', index=False)

In [25]:
combined_df = pd.concat([gazeta_df, podrobno_df, uzreport_df], ignore_index=True)

combined_df.shape

(30730, 2)

In [26]:
combined_df.head()

,text,sentiment
0,o‘zbekiston futbol assotsiatsiyasi intizom qo‘mitasi bunyodkor va nasaffutbolchilari o‘rtasidagi ziddiyatni o‘rganmoqda,positive
1,rustam yusupov toshkent obodonlashtirish bosh boshqarmasi rahbari etib tayinlandi yanvar oyi oxirida anvar umarov ishdan bo‘shatilgandan keyin bu lavozim bo‘sh qoldi,positive
2,xitoyning yutong kompaniyasi yil oxirigacha samarqandga ta elektr avtobus yetkazib beradi shu jumladan yilning birinchi yarmida ta va uchinchi chorak oxirigacha ta hammasi bo‘lib ta elektr avtobus sotib olish rejalashtirilgan shaharda elektr avtobuslari uchun depo ham qurilishi rejalashtirilgan,positive
3,samarqand viloyatida gektar o‘rmon xo‘jaligi yerlarini ming dollarga sotishga uringan sotuvchi ming dollar avans olganida hibsga olingan,positive
4,andijon viloyati xonobodda bir kishi davlat organi xodimini pichoq bilan jarohatladi qasddan qotillikka urinishmoddasi bo‘yicha jinoiy ish qo‘zg‘atildi,negative


In [27]:
# shuffle the rows
combined_df = combined_df.sample(frac=1).reset_index(drop=True)
combined_df.head()

,text,sentiment
0,milliy elektron ommaviy axborot vositalari assotsiatsiyasi naesmi o‘z nomini o‘zgartirdi endi bu o‘zbekiston milliy media assotsiatsiyasi mass media jamg‘armasi vasiylik kengashi raisi komil allamjonov ommaviy axborot vositalari vakillarini matbuotni birlashtirishning yangi nuqtasi yaratilgani bilan tabriklab nihoyat bu,positive
1,wizzair abu dabi arzon aviakompaniyasi abu dabi samarqand abu dabi yo‘nalishi bo‘yicha qo‘shimcha chastotani taqdim etadi,positive
2,yildan beri yuvinmagan dunyoning eng kir odami yoshida vafot etdi u haqida tasavvurga sig‘dirish ilojsiz bo‘lgan ma‘lumotlar yillab cho‘milmaganining sababi hayot tarzi va nega bunday yashagani haqida qiziqarli ma‘lumotlar va fotolarni ushbu xabarda o‘qishingiz mumkin,negative
3,o‘zbekiston birinchi kuzgi sovuqni kutmoqda dushanba kuni kechasi tungi harorat darajaga etadi kunduzgi harorat esa darajaga tushadi,positive
4,iyundan tibbiyot xodimlarining oylik maoshi oshiriladi,positive


In [28]:
combined_df.shape

(30730, 2)

In [29]:
# check one more time for empty strings
print((combined_df['text'].str.strip().str.len() == 0).sum())

0


In [30]:
combined_df['length'] = combined_df['text'].str.len()
print(combined_df['length'].describe())

count    30730.000000
mean       169.171266
std        106.156526
min          7.000000
25%         87.000000
50%        129.000000
75%        229.000000
max        456.000000
Name: length, dtype: float64


In [31]:
combined_df = combined_df[['text', 'sentiment']]

In [32]:
combined_df.to_csv('combined_uznews_30k_preprocessed_ready_for_modeling.csv', index=False)